# Airline Route Optimization: Data-Driven Decision Making for Market Entry

In [1]:
# Data Handling & Preprocessing
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Processing & Transformation
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Statistical Analysis
import scipy.stats as stats

# File Handling & System Operations
import os

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

# Display settings for Pandas
pd.set_option("display.float_format", "{:.2f}".format)

print("Libraries imported successfully!")

Libraries imported successfully!


### Load the data in python Env.

In [2]:
# Define file paths
flights_file = "Flights.csv"
tickets_file = "Tickets.csv"
airport_codes_file = "Airport_Codes.csv"

# Load datasets into Pandas DataFrames
flights_df = pd.read_csv(flights_file)
tickets_df = pd.read_csv(tickets_file)
airport_codes_df = pd.read_csv(airport_codes_file)


#### What are the 10 busiest round-trip routes in terms of the number of round-trip flights in Q1 2019?

In [3]:
flights_df.columns

Index(['FL_DATE', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', 'DEST_AIRPORT_ID',
       'DESTINATION', 'DEST_CITY_NAME', 'DEP_DELAY', 'ARR_DELAY', 'CANCELLED',
       'AIR_TIME', 'DISTANCE', 'OCCUPANCY_RATE'],
      dtype='object')

In [4]:
if 'Cancelled' in flights_df.columns:
    valid_flights = flights_df[flights_df['Cancelled'] == 0]  # Assuming 0 means NOT canceled
elif 'Flight_Status' in flights_df.columns:
    valid_flights = flights_df[flights_df['Flight_Status'] == 'Completed']
else:
    valid_flights = flights_df  # If no column exists, use the full dataset


In [5]:
# Filter out canceled flights (CANCELLED = 0 means flight was not canceled)
valid_flights = flights_df[flights_df['CANCELLED'] == 0]

In [6]:
# Count flights in each direction (Origin → Destination and Destination → Origin)
round_trip_counts = valid_flights.groupby(['ORIGIN', 'DESTINATION']).size().reset_index(name='Flight_Count')


In [7]:
# Count return flights (Destination → Origin)
round_trip_counts_reversed = valid_flights.groupby(['DESTINATION', 'ORIGIN']).size().reset_index(name='Flight_Count_Reversed')


In [8]:
# Rename columns for merging
round_trip_counts_reversed.columns = ['ORIGIN', 'DESTINATION', 'Flight_Count_Reversed']


In [9]:
# Merge both flight counts
merged_counts = round_trip_counts.merge(round_trip_counts_reversed, on=['ORIGIN', 'DESTINATION'], how='left')


In [10]:
# Sum both directions to get total round-trip flights
merged_counts['Total_Round_Trips'] = merged_counts['Flight_Count'] + merged_counts['Flight_Count_Reversed']


In [11]:
# Get the top 10 busiest round-trip routes
top_10_busiest_routes = merged_counts.nlargest(10, 'Total_Round_Trips')

In [12]:
# Display results
print("Top 10 Busiest Round-Trip Routes:")
top_10_busiest_routes

Top 10 Busiest Round-Trip Routes:


,ORIGIN,DESTINATION,Flight_Count,Flight_Count_Reversed,Total_Round_Trips
3010,LAX,SFO,4164,4176.00,8340.00
5284,SFO,LAX,4176,4164.00,8340.00
3115,LGA,ORD,3576,3580.00,7156.00
4066,ORD,LGA,3580,3576.00,7156.00
2869,LAS,LAX,3254,3257.00,6511.00
2971,LAX,LAS,3257,3254.00,6511.00
2742,JFK,LAX,3158,3162.00,6320.00
2969,LAX,JFK,3162,3158.00,6320.00
3009,LAX,SEA,2502,2497.00,4999.00
5137,SEA,LAX,2497,2502.00,4999.00


What are the 10 most profitable round-trip routes (without considering the upfront airplane cost) in Q1 2019?

 - Show total revenue, total cost, summary values of key components, and total round-trip flights for the top 10 most profitable routes. (Exclude canceled flights.)

In [13]:
# Ensure 'DISTANCE' is numeric (convert if necessary)
flights_df['DISTANCE'] = pd.to_numeric(flights_df['DISTANCE'], errors='coerce')

In [14]:
# Filter out canceled flights (CANCELLED = 0 means flight was not canceled)
valid_flights = flights_df[flights_df['CANCELLED'] == 0].copy()  # Use .copy() to avoid SettingWithCopyWarning

In [15]:
# Define cost components (as per challenge instructions)
fuel_maintenance_crew_cost_per_mile = 8.0  # $ per mile
depreciation_insurance_cost_per_mile = 1.18  # $ per mile

In [16]:
# Compute cost per flight (ensuring 'DISTANCE' is properly converted)
valid_flights.loc[:, 'Flight_Cost'] = valid_flights['DISTANCE'].astype(float) * (fuel_maintenance_crew_cost_per_mile + depreciation_insurance_cost_per_mile)

In [17]:
# Display sample output to verify
valid_flights[['DISTANCE', 'Flight_Cost']].head(10)

,DISTANCE,Flight_Cost
0,1025.00,9409.50
1,930.00,8537.40
2,930.00,8537.40
3,930.00,8537.40
4,1005.00,9225.90
5,1147.00,10529.46
6,945.00,8675.10
7,945.00,8675.10
8,945.00,8675.10
9,945.00,8675.10


#### Which 5 round-trip routes do you recommend investing in? (Based on factors you choose.)

In [18]:
# Ensure necessary columns are numeric
flights_df['DISTANCE'] = pd.to_numeric(flights_df['DISTANCE'], errors='coerce')
flights_df['DEP_DELAY'] = pd.to_numeric(flights_df['DEP_DELAY'], errors='coerce')
flights_df['ARR_DELAY'] = pd.to_numeric(flights_df['ARR_DELAY'], errors='coerce')
flights_df['OCCUPANCY_RATE'] = pd.to_numeric(flights_df['OCCUPANCY_RATE'], errors='coerce')

In [19]:
# Filter out canceled flights
valid_flights = flights_df[flights_df['CANCELLED'] == 0].copy()

In [20]:
# Define cost components
fuel_maintenance_crew_cost_per_mile = 8.0  # $ per mile
depreciation_insurance_cost_per_mile = 1.18  # $ per mile
delay_cost_per_minute = 75.0  # Additional cost per minute after 15 min delay

In [21]:
# Compute revenue per flight
ticket_price_per_passenger = valid_flights['OCCUPANCY_RATE'] * 200  # Assuming ticket price * occupancy
baggage_fee_per_passenger = 70 * 0.5  # 50% of passengers check bags

In [22]:
valid_flights['Revenue'] = (ticket_price_per_passenger * 200) + (baggage_fee_per_passenger * 200)

In [23]:
# Compute cost per flight
valid_flights['Flight_Cost'] = valid_flights['DISTANCE'] * (fuel_maintenance_crew_cost_per_mile + depreciation_insurance_cost_per_mile)

In [24]:
# Compute delay cost
valid_flights['DEP_DELAY'] = valid_flights['DEP_DELAY'].clip(lower=0) - 15  # Remove first 15 minutes
valid_flights['ARR_DELAY'] = valid_flights['ARR_DELAY'].clip(lower=0) - 15  # Remove first 15 minutes

In [25]:
# Ensure no negative delay times
valid_flights['DEP_DELAY'] = valid_flights['DEP_DELAY'].clip(lower=0)
valid_flights['ARR_DELAY'] = valid_flights['ARR_DELAY'].clip(lower=0)

In [26]:
# Compute total delay cost
valid_flights['Delay_Cost'] = (valid_flights['DEP_DELAY'] + valid_flights['ARR_DELAY']) * delay_cost_per_minute

In [27]:
# Compute total cost including delays
valid_flights['Total_Cost'] = valid_flights['Flight_Cost'] + valid_flights['Delay_Cost']

In [28]:
# Compute profit per flight
valid_flights['Profit'] = valid_flights['Revenue'] - valid_flights['Total_Cost']

In [29]:
# Aggregate data at the round-trip route level
route_analysis = valid_flights.groupby(['ORIGIN', 'DESTINATION']).agg(
    Total_Revenue=('Revenue', 'sum'),
    Total_Cost=('Total_Cost', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Total_Round_Trips=('ORIGIN', 'count'),  # Counting flights per route
    Avg_Occupancy=('OCCUPANCY_RATE', 'mean'),
    Avg_Dep_Delay=('DEP_DELAY', 'mean'),
    Avg_Arr_Delay=('ARR_DELAY', 'mean')
).reset_index()

In [30]:
# Normalize delay times (lower is better) and profit (higher is better)
route_analysis['Score'] = (
    (route_analysis['Total_Profit'] / route_analysis['Total_Profit'].max()) * 0.5 +  # Profitability weight
    (route_analysis['Avg_Occupancy'] / route_analysis['Avg_Occupancy'].max()) * 0.3 +  # Occupancy weight
    ((route_analysis['Avg_Dep_Delay'].max() - route_analysis['Avg_Dep_Delay']) / route_analysis['Avg_Dep_Delay'].max()) * 0.1 +  # Departure delay weight
    ((route_analysis['Avg_Arr_Delay'].max() - route_analysis['Avg_Arr_Delay']) / route_analysis['Avg_Arr_Delay'].max()) * 0.1  # Arrival delay weight
)

In [31]:
# Select the top 5 routes for investment
top_5_routes = route_analysis.nlargest(5, 'Score')

In [32]:
# Display results
print("Top 5 Recommended Round-Trip Routes:")
top_5_routes

Top 5 Recommended Round-Trip Routes:


,ORIGIN,DESTINATION,Total_Revenue,Total_Cost,Total_Profit,Total_Round_Trips,Avg_Occupancy,Avg_Dep_Delay,Avg_Arr_Delay,Score
5284,SFO,LAX,138306800.00,23083855.50,115187544.50,4176,0.65,15.34,17.16,0.89
3010,LAX,SFO,136584000.00,24737588.28,111650811.72,4164,0.65,18.28,19.80,0.87
2971,LAX,LAS,107763400.00,11493775.92,96163824.08,3257,0.65,9.06,9.14,0.81
2869,LAS,LAX,106949200.00,11527842.96,95332557.04,3254,0.65,8.82,9.55,0.80
4066,ORD,LGA,118072800.00,35118010.50,82787789.50,3580,0.65,19.48,21.76,0.74


#### How many round-trip flights are required to break even on the upfront airplane cost for each of the 5 recommended routes? (Include key summary components.)

In [ ]:
# Ensure necessary columns are numeric
flights_df['DISTANCE'] = pd.to_numeric(flights_df['DISTANCE'], errors='coerce')
flights_df['DEP_DELAY'] = pd.to_numeric(flights_df['DEP_DELAY'], errors='coerce')
flights_df['ARR_DELAY'] = pd.to_numeric(flights_df['ARR_DELAY'], errors='coerce')
flights_df['OCCUPANCY_RATE'] = pd.to_numeric(flights_df['OCCUPANCY_RATE'], errors='coerce')

In [ ]:
# Filter out canceled flights
valid_flights = flights_df[flights_df['CANCELLED'] == 0].copy()

In [ ]:
# Define cost components
fuel_maintenance_crew_cost_per_mile = 8.0  # $ per mile
depreciation_insurance_cost_per_mile = 1.18  # $ per mile
delay_cost_per_minute = 75.0  # Additional cost per minute after 15 min delay
airplane_cost = 90_000_000  # $90 million per airplane

In [ ]:
# Compute revenue per flight
ticket_price_per_passenger = valid_flights['OCCUPANCY_RATE'] * 200  # Assuming ticket price * occupancy
baggage_fee_per_passenger = 70 * 0.5  # 50% of passengers check bags

In [ ]:
valid_flights['Revenue'] = (ticket_price_per_passenger * 200) + (baggage_fee_per_passenger * 200)

In [ ]:
# Compute cost per flight
valid_flights['Flight_Cost'] = valid_flights['DISTANCE'] * (fuel_maintenance_crew_cost_per_mile + depreciation_insurance_cost_per_mile)

In [ ]:
# Compute delay cost
valid_flights['DEP_DELAY'] = valid_flights['DEP_DELAY'].clip(lower=0) - 15  # Remove first 15 minutes
valid_flights['ARR_DELAY'] = valid_flights['ARR_DELAY'].clip(lower=0) - 15  # Remove first 15 minutes
valid_flights['DEP_DELAY'] = valid_flights['DEP_DELAY'].clip(lower=0)
valid_flights['ARR_DELAY'] = valid_flights['ARR_DELAY'].clip(lower=0)

In [ ]:
# Compute total delay cost
valid_flights['Delay_Cost'] = (valid_flights['DEP_DELAY'] + valid_flights['ARR_DELAY']) * delay_cost_per_minute

In [ ]:
# Compute total cost including delays
valid_flights['Total_Cost'] = valid_flights['Flight_Cost'] + valid_flights['Delay_Cost']

In [ ]:
# Compute profit per flight
valid_flights['Profit'] = valid_flights['Revenue'] - valid_flights['Total_Cost']

In [ ]:
# Aggregate data at the round-trip route level
route_analysis = valid_flights.groupby(['ORIGIN', 'DESTINATION']).agg(
    Total_Revenue=('Revenue', 'sum'),
    Total_Cost=('Total_Cost', 'sum'),
    Total_Profit=('Profit', 'sum'),
    Total_Round_Trips=('ORIGIN', 'count'),  # Counting flights per route
    Avg_Occupancy=('OCCUPANCY_RATE', 'mean'),
    Avg_Dep_Delay=('DEP_DELAY', 'mean'),
    Avg_Arr_Delay=('ARR_DELAY', 'mean')
).reset_index()

In [ ]:
# Normalize delay times (lower is better) and profit (higher is better)
route_analysis['Score'] = (
    (route_analysis['Total_Profit'] / route_analysis['Total_Profit'].max()) * 0.5 +  # Profitability weight
    (route_analysis['Avg_Occupancy'] / route_analysis['Avg_Occupancy'].max()) * 0.3 +  # Occupancy weight
    ((route_analysis['Avg_Dep_Delay'].max() - route_analysis['Avg_Dep_Delay']) / route_analysis['Avg_Dep_Delay'].max()) * 0.1 +  # Departure delay weight
    ((route_analysis['Avg_Arr_Delay'].max() - route_analysis['Avg_Arr_Delay']) / route_analysis['Avg_Arr_Delay'].max()) * 0.1  # Arrival delay weight
)

In [ ]:
# Select the top 5 routes for investment
top_5_routes = route_analysis.nlargest(5, 'Score')

In [ ]:
# Compute Break-Even Flights for each route
top_5_routes['Profit_per_Round_Trip'] = top_5_routes['Total_Profit'] / top_5_routes['Total_Round_Trips']
top_5_routes['Break_Even_Flights'] = airplane_cost / top_5_routes['Profit_per_Round_Trip']

In [34]:
# Display the top 5 recommended routes with break-even calculations
top_5_routes[['ORIGIN', 'DESTINATION', 'Total_Profit', 'Total_Round_Trips', 'Profit_per_Round_Trip', 'Break_Even_Flights']]

,ORIGIN,DESTINATION,Total_Profit,Total_Round_Trips,Profit_per_Round_Trip,Break_Even_Flights
5284,SFO,LAX,115187544.50,4176,27583.22,3262.85
3010,LAX,SFO,111650811.72,4164,26813.36,3356.54
2971,LAX,LAS,96163824.08,3257,29525.28,3048.24
2869,LAS,LAX,95332557.04,3254,29297.04,3071.98
4066,ORD,LGA,82787789.50,3580,23125.08,3891.88
